# Functional Usage

In [1]:
from braandket_circuit import CNOT, FreezePass, H, M, QOperation, Sequential, allocate_qubits, compile

As we already know, a circuit can be defined using `Sequential`, `.on()` and some operations.

In [2]:
circuit = Sequential([
    H.on(0),
    CNOT.on(0, 1),
    M
])


If you feel annoying pointing out target qubit using `.on()` and indices, do try functional usage!

In functional usage, we can directly call `QOperation` instances like functions with qubits as arguments. In most cases, functional usage is more convenient and readable.

Code below performs the whole circuit directly, without `Sequential` and `.on()`. As we can see, calling `M` returns a `MeasurementResult` instance.


In [3]:
q0, q1 = allocate_qubits(2)

H(q0)
CNOT(q0, q1)
result = M(q0, q1)

print(f"{result.value=}")
print(f"{result.prob=}")

result.value=array([0, 0], dtype=int64)
result.prob=array(0.5)


You may want to wrap the circuit up, distinguishing it from other parts. Then you'll find that the circuit can be defined as a function (That's why it is called "functional").

In [4]:
# noinspection PyRedeclaration
def circuit(q0, q1):
    H(q0)
    CNOT(q0, q1)
    return M(q0, q1)


q0, q1 = allocate_qubits(2)
result = circuit(q0, q1)

print(f"{result.value=}")
print(f"{result.prob=}")

result.value=array([0, 0], dtype=int64)
result.prob=array(0.5)


Furthermore, you can define your circuit as a subclass of `QOpeartion` by overriding the `__call__` method.

In [5]:
class MyCircuit(QOperation):
    def __call__(self, q0, q1):
        H(q0)
        CNOT(q0, q1)
        return M(q0, q1)


q0, q1 = allocate_qubits(2)

circuit = MyCircuit()
result = circuit(q0, q1)

print(f"{result.value=}")

print(f"{result.prob=}")

result.value=array([0, 0], dtype=int64)
result.prob=array(0.5)


Compared to defining a plain function, subclassing `QOperation` enables more features:

* Instances `QOperation` have commonly used methods and properties, like `.on()` and `name`.
* You can define additional methods and properties in the subclass of `QOperation`.
* Instances of `QOperation` is recognized for compilation, optimization, visualization, execution etc.

Compared to constructing a `Sequential` instance, defining a circuit as a function or (subclass with `__call__`) is very different: the function is executed dynamically when you call it, while the instance of `Sequential` is static. They all have their pros and cons.
* A dynamic circuit can be useful for debugging. For example, you can set a breakpoint in the function and inspect the intermediate quantum state.
* A static circuit can be useful for compilation, optimization, visualization. Because the compiler (optimizer, visualizer) requires a static circuit to work on.

A conversion from dynamic circuit to static circuit is supported by a special compiling process `FreezePass`. Such conversion is performed by "tracing". The compiler calls the operation with symbolic arguments, traces all the callings of the sub-operations and construct the static circuit according to the records. (Due to such principle, the conversion has some limitations.) An example is shown below. 

In [6]:
frozen = compile(FreezePass(), circuit)
print(frozen)

Sequential([
	Remapped(HadamardGate(), 0),
	Remapped(Controlled(PauliXGate()), 0, 1),
	Remapped(ProjectiveMeasurement(), 0, 1),
])
